1.a. I've used another csv file for copying the entire data and to add the new field "highCrime" to avoid complications. 

In [18]:
import csv
reader=csv.reader(open('communities-crime-clean.csv','rU'))
writer=csv.writer(open('outputSample.csv','wb'))
i=0
for row in reader:
    if(i==0):
        row.append("highCrime")
        i=1
    if(row[103]!='ViolentCrimesPerPop'):
        if(float(row[103])>0.1):
            row.append("true")
        else:
            row.append("false")
    writer.writerow(row)

totalCount=0
trueCount=0
falseCount=0

tempRead=csv.reader(open('outputSample.csv','r'))

for row0 in tempRead:
    if(row0[104]=='true'):
        trueCount=trueCount+1
        totalCount=totalCount+1
    if(row0[104]=='false'):
        falseCount=falseCount+1
        totalCount=totalCount+1

truePercent=100
falsePercent=100
truePercent=(trueCount/totalCount)
falsePercent=100*float(falseCount/totalCount)
print("TrueCount = ",trueCount)
print("FalseCount = ",falseCount)
print("TotalCount = ",totalCount)
print(truePercent)
print("Percentage of (highCrime=true) = ",truePercent,"%")
print("Percentage of (highCrime=false) = ",falsePercent,"%")

('TrueCount = ', 1250)
('FalseCount = ', 743)
('TotalCount = ', 1993)
0
('Percentage of (highCrime=true) = ', 0, '%')
('Percentage of (highCrime=false) = ', 0.0, '%')


1.b. Using the DecisionTreeClassifier module

In [40]:
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn import tree

data=pd.read_csv('C:/CrimePredictionData/outputSample.csv',header=0)

print ("Dataset Length: ", len(data))
print ("Dataset Shape:: ", data.shape)

Xvalues=data.values[:,3:103]
Yvalues=data.values[:,104]

for i in range(len(Yvalues)):
    Yvalues[i]=str(Yvalues[i])

X_train, X_test, Y_train, Y_test = train_test_split(Xvalues,Yvalues,test_size = 0.3)

dTreeClass = DecisionTreeClassifier(criterion = "gini", random_state = 598)
print(dTreeClass)

#print(Y_train)
Y_training=list()
for val in Y_train:
    Y_training.append(val)

#X_train=np.array(X_train).reshape(-1,1)
#Y_training=Y_training.reshape(-1,1)
y_training=np.asarray(Y_training)
y_training=y_training.reshape(-1,1)
dTreeClass=dTreeClass.fit(X_train,y_training)

#X_test=np.array(X_test).reshape((-1,1))
y_pred=dTreeClass.predict(X_test)

print("Accuracy = ",accuracy_score(Y_test,y_pred)*100,"%")
print("Precision = ",precision_score(Y_test,y_pred,pos_label="True")*100,"%")
print("Recall = ",recall_score(Y_test,y_pred,pos_label="True")*100,"%")
importances = dTreeClass.feature_importances_
indices = np.argsort(importances)[::-1]

#print("Most important feature = feature ",importances[0])

print("Feature ranking:")

for f in range(Xvalues.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))

print("Most important feature is : feature ",indices[0])


Dataset Length:  1993
Dataset Shape::  (1993, 105)
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=598, splitter='best')
Accuracy =  77.2575250836 %
Precision =  82.8205128205 %
Recall =  82.3979591837 %
Feature ranking:
1. feature 44 (0.352464)
2. feature 3 (0.084919)
3. feature 5 (0.057541)
4. feature 71 (0.020375)
5. feature 32 (0.020074)
6. feature 47 (0.019796)
7. feature 78 (0.019442)
8. feature 33 (0.019381)
9. feature 46 (0.018322)
10. feature 22 (0.017254)
11. feature 45 (0.015575)
12. feature 38 (0.013152)
13. feature 12 (0.011717)
14. feature 25 (0.011295)
15. feature 23 (0.010816)
16. feature 61 (0.010635)
17. feature 0 (0.010594)
18. feature 39 (0.009957)
19. feature 65 (0.009119)
20. feature 72 (0.009055)
21. feature 85 (0.008782)
22. 

1b.
(i) Accuracy =  77.7591973244 %
Precision =  83.9050131926 %
Recall =  81.5384615385 %

(ii) The most important feature used for classifying is PctKids2Par, which is the percentage of kids in family housing with 2 parents. This seems pretty logical as the percentage of kids increases in a family, the likelihood of crimes can go up.

1c. Applying cross_val_score to do 10-fold cross-validation

In [41]:
from sklearn import linear_model
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer

crossValScoreAccuracy=cross_val_score(dTreeClass, Xvalues, Yvalues,cv=10)
print("Mean Accuracy = ",np.mean(crossValScoreAccuracy))

precisionScorer = make_scorer(precision_score, pos_label='True')
crossValScorePrecision=cross_val_score(dTreeClass,Xvalues,Yvalues,cv=10,scoring=precisionScorer)
print("Mean Precision = ",np.mean(crossValScorePrecision))

recallScorer=make_scorer(recall_score,pos_label='True')
crossValScoreRecall=cross_val_score(dTreeClass,Xvalues,Yvalues,cv=10,scoring=recallScorer)
print("Mean Recall = ",np.mean(crossValScoreRecall))

Mean Accuracy =  0.719479899497
Mean Precision =  0.781190893925
Mean Recall =  0.7688


1c.
(i) The metrics of the 10-fold cross-validation are :
Mean Accuracy =  0.725979899497
Mean Precision =  0.788705794886
Mean Recall =  0.7712

(ii) The difference between these values and the previous results is due to the sample taken for computation. In the previous case, we took 30% data for testing. But in this case, we are taking 10-folds data and computing the metrics for each fold and then we take the average.

2a. Using GaussianNB to learn a Naive Bayes classifier to predict highCrime

In [67]:
from sklearn.naive_bayes import GaussianNB
nBayesClass=GaussianNB()
nBayesClass.fit(X_train,Y_train)
GaussianNB(priors=None)
y_predBayes=nBayesClass.predict(X_test)
#print("Predicted values : ",y_predBayes)

nbAccuracy=cross_val_score(nBayesClass,Xvalues,Yvalues,cv=10)
print("Naive Bayes mean accuracy = ",np.mean(nbAccuracy))

nbPrecision=cross_val_score(nBayesClass,Xvalues,Yvalues,cv=10,scoring=precisionScorer)
print("Naive Bayes mean precision = ",np.mean(nbPrecision))

nbRecall=cross_val_score(nBayesClass,Xvalues,Yvalues,cv=10,scoring=recallScorer)
print("Naive Bayes mean recall = ",np.mean(nbRecall))

normAbsList=[]
for i in range(Xvalues.shape[1]):
    sumTrue=0.0
    sumFalse=0.0
    meanTrue=0.0
    meanFalse=0.0
    itr=0
    countTrue=0
    countFalse=0
    for val in Xvalues[:,i]:
        if(str(Yvalues.item(itr))=='True'):
            sumTrue=sumTrue+val
            countTrue=countTrue+1
            itr=itr+1
        else:
            sumFalse=sumFalse+val
            countFalse=countFalse+1
            itr=itr+1
    meanTrue=sumTrue/countTrue
    meanFalse=sumFalse/countFalse
    itr=0
    sumSquareTrue=0.0
    sumSquareFalse=0.0
    for val1 in Xvalues[:,i]:
        if(str(Yvalues.item(itr))=='True'):
            sumSquareTrue=sumSquareTrue+((val1-meanTrue)*(val1-meanTrue))
            itr=itr+1
        else:
            sumSquareFalse=sumSquareFalse+((val1-meanFalse)*(val1-meanFalse))
            itr=itr+1
    sdTrue=(sumSquareTrue/countTrue)**(1/2.0)
    sdFalse=(sumSquareFalse/countFalse)**(1/2.0)
    normAbsDif=(abs(meanTrue-meanFalse))/(sdTrue+sdFalse)
    normAbsList.append(normAbsDif)
    sortedIndices=np.argsort(normAbsList)[::-1]

print("Features and the differences : ")
for x in range(10):
    print("Feature ",sortedIndices[x]," difference = ",normAbsList[sortedIndices[x]])

Naive Bayes mean accuracy =  0.761608040201
Naive Bayes mean precision =  0.911799814828
Naive Bayes mean recall =  0.692
Features and the differences : 
Feature  44  difference =  0.8097484282277082
Feature  43  difference =  0.7455448082166711
Feature  3  difference =  0.735229953123246
Feature  50  difference =  0.7092610512005777
Feature  40  difference =  0.6939780883033155
Feature  41  difference =  0.6746446092224765
Feature  45  difference =  0.665008569884773
Feature  15  difference =  0.6610764326654875
Feature  46  difference =  0.6429494481747667
Feature  38  difference =  0.6168636592230385


2a.
(i) The metrics of the 10-fold cross-validation are,
    Naive Bayes mean accuracy =  0.761608040201
    Naive Bayes mean precision =  0.911799814828
    Naive Bayes mean recall =  0.692

(ii) The 10 most predictive features are :
    PctKids2Par, PctFam2Par, population, PctIlleg, FemalePctDiv, TotalPctDiv, PctYoungKids2Par, pctWInvInc, PctTeen2Par and MalePctDivorce
    The larger the difference, the more predictive the feature is. This is because as the difference becomes large, there's a huge disparity between the values that correspond to the True class and False class. The class label can be easily determined as the difference gets large.

(iii) These results are slightly different from that obtained from the Decision Tree. This is because of the difference in how both the models learn from the data.

2b. Implementing LinearSVC :

In [92]:
from sklearn import svm
linearSvcModel=svm.LinearSVC()
linearSvcModel.fit(Xvalues,Yvalues)
svm.LinearSVC()

svcAccuracy=cross_val_score(linearSvcModel,Xvalues,Yvalues,cv=10)
print("Linear SVC accuracy = ",np.mean(svcAccuracy))

svcPrecision=cross_val_score(linearSvcModel,Xvalues,Yvalues,cv=10,scoring=precisionScorer)
print("Linear SVC precision = ",np.mean(svcPrecision))

svcRecall=cross_val_score(linearSvcModel,Xvalues,Yvalues,cv=10,scoring=recallScorer)
print("Linear SVC recall = ",np.mean(svcRecall))

absFeatureWts=abs(linearSvcModel.coef_)
index=np.argsort(absFeatureWts)[::-1]

absFeatureWts

Linear SVC accuracy =  0.796233668342
Linear SVC precision =  0.845404856531
Linear SVC recall =  0.8344


array([[ 1.00230082,  0.64937299,  0.66921797,  1.50021955,  0.03286906,
         0.77030062,  0.98827289,  0.75968963,  0.01963456,  0.890573  ,
         0.03139657,  0.13348688,  0.39527268,  0.33962798,  0.20031268,
         1.88848756,  0.09050429,  0.25781507,  0.07894048,  0.41782071,
         0.28464109,  0.1731376 ,  0.05577263,  0.05690586,  0.10881711,
         0.39363079,  0.18841948,  1.05154795,  0.32272802,  0.21534113,
         0.0918559 ,  0.54218142,  0.22415196,  0.43352192,  0.1550712 ,
         0.46510799,  0.00259216,  1.01467415,  1.06569569,  0.66952685,
         0.20033822,  0.19165646,  0.1824921 ,  0.86980893,  1.19032855,
         0.20274708,  0.37962831,  0.04120569,  0.08783801,  0.64728588,
         0.04921386,  0.42902907,  0.31035694,  0.41097297,  0.21505436,
         0.45430688,  0.26380772,  0.57995721,  0.02945877,  0.7137713 ,
         0.49697695,  0.10324896,  0.36563778,  0.95739292,  1.75512634,
         0.04692473,  0.27352158,  0.19317892,  0.7

2b.
(i) The metrics from 10-fold cross-validation are:
Linear SVC accuracy =  0.796233668342
Linear SVC precision =  0.845404856531
Linear SVC recall =  0.8344

(ii) The 10 most predictive features are:

pctWInvInc, PersPerOccupHous, racePctWhite, PctKids2Par, RentHighQ, MalePctDivorce, NumUnderPov, NumStreet, PctOccupMgmtProf, population

(iii) These results are slightly different from the decision tree as the methods involved in learning by both the models are different.

3a. Implementing LinearRegression to predict the crime rate per capita.

In [122]:
from sklearn import linear_model
YregVal=data.values[:,103]

linReg=linear_model.LinearRegression()
linReg.fit(Xvalues,YregVal)
linear_model.LinearRegression()

YregPred=linReg.predict(Xvalues)
mse=cross_val_score(linReg,Xvalues,YregVal,cv=10,scoring='neg_mean_squared_error')
print("Estimated MSE of 10-fold cross-validation = ",np.mean(mse))

mseEntire=cross_val_score(linReg,Xvalues,YregPred,cv=10,scoring='neg_mean_squared_error')
print("Estimated MSE of training set = ",np.mean(mseEntire))

meanCoeff=np.mean(linReg.coef_)
coeffList=[]
for val in linReg.coef_:
    coeffList.append((val-meanCoeff)/np.std(linReg.coef_))
    
coefIndex=np.argsort(coeffList)[::-1]
for i in range(10):
    print("Feature ",coefIndex[i],"'s importance = ",coeffList[coefIndex[i]])

Estimated MSE of 10-fold cross-validation =  -0.0200939693044
Estimated MSE of training set =  -1.51827513156e-30
Feature  64 's importance =  3.31503009227
Feature  73 's importance =  2.96604077521
Feature  38 's importance =  2.39468803667
Feature  58 's importance =  2.25913430374
Feature  85 's importance =  1.94752839434
Feature  19 's importance =  1.5057912585
Feature  33 's importance =  1.29988149452
Feature  39 's importance =  1.18653263332
Feature  68 's importance =  1.12202855573
Feature  80 's importance =  1.1143315441


3a.

(i) Estimated MSE of 10-fold cross-validation =  -0.0200939693044

(ii) Estimated MSE of training set =  -1.47277662963e-30

(iii) The 10 most important features are :
        PersPerOccupHouse, PctHousOwnOcc, MalePctDivorce, PctRecImmig8, MedRent, medFamInc, PctEmploy, MalePctNevMarr, PctPersDenseHous, OwnOccMedVal

3b. Using Ridge

In [142]:
ridReg=linear_model.Ridge(alpha=10)
ridReg.fit(Xvalues,YregVal)
yRidgePred=ridReg.predict(Xvalues)
mse=cross_val_score(ridReg,Xvalues,YregVal,cv=10,scoring='neg_mean_squared_error')
print("Estimated MSE of 10-fold CV (alpha=10) = ",np.mean(mse))
mse=cross_val_score(ridReg,Xvalues,yRidgePred,cv=10,scoring='neg_mean_squared_error')
print("Estimated MSE of training data (alpha=10) = ",np.mean(mse))
print()

ridReg=linear_model.Ridge(alpha=1)
ridReg.fit(Xvalues,YregVal)
yRidgePred=ridReg.predict(Xvalues)
mse=cross_val_score(ridReg,Xvalues,YregVal,cv=10,scoring='neg_mean_squared_error')
print("Estimated MSE of 10-fold CV (alpha=1) = ",np.mean(mse))
mse=cross_val_score(ridReg,Xvalues,yRidgePred,cv=10,scoring='neg_mean_squared_error')
print("Estimated MSE of training data (alpha=1) = ",np.mean(mse))
print()

ridReg=linear_model.Ridge(alpha=0.1)
ridReg.fit(Xvalues,YregVal)
yRidgePred=ridReg.predict(Xvalues)
mse=cross_val_score(ridReg,Xvalues,YregVal,cv=10,scoring='neg_mean_squared_error')
print("Estimated MSE of 10-fold CV (alpha=0.1) = ",np.mean(mse))
mse=cross_val_score(ridReg,Xvalues,yRidgePred,cv=10,scoring='neg_mean_squared_error')
print("Estimated MSE of training data (alpha=0.1) = ",np.mean(mse))
print()

ridReg=linear_model.Ridge(alpha=0.01)
ridReg.fit(Xvalues,YregVal)
yRidgePred=ridReg.predict(Xvalues)
mse=cross_val_score(ridReg,Xvalues,YregVal,cv=10,scoring='neg_mean_squared_error')
print("Estimated MSE of 10-fold CV (alpha=0.01) = ",np.mean(mse))
mse=cross_val_score(ridReg,Xvalues,yRidgePred,cv=10,scoring='neg_mean_squared_error')
print("Estimated MSE of training data (alpha=0.01) = ",np.mean(mse))
print()

ridReg=linear_model.Ridge(alpha=0.001)
ridReg.fit(Xvalues,YregVal)
yRidgePred=ridReg.predict(Xvalues)
mse=cross_val_score(ridReg,Xvalues,YregVal,cv=10,scoring='neg_mean_squared_error')
print("Estimated MSE of 10-fold CV (alpha=0.001) = ",np.mean(mse))
mse=cross_val_score(ridReg,Xvalues,yRidgePred,cv=10,scoring='neg_mean_squared_error')
print("Estimated MSE of training data (alpha=0.001) = ",np.mean(mse))
print()


Estimated MSE of 10-fold CV (alpha=10) =  -0.0200342497979
Estimated MSE of training data (alpha=10) =  -0.000152996873221

Estimated MSE of 10-fold CV (alpha=1) =  -0.0197950213482
Estimated MSE of training data (alpha=1) =  -8.73728120217e-05

Estimated MSE of 10-fold CV (alpha=0.1) =  -0.01991972494
Estimated MSE of training data (alpha=0.1) =  -1.63387410814e-05

Estimated MSE of 10-fold CV (alpha=0.01) =  -0.0200559556275
Estimated MSE of training data (alpha=0.01) =  -1.31270726501e-06

Estimated MSE of 10-fold CV (alpha=0.001) =  -0.020089527318
Estimated MSE of training data (alpha=0.001) =  -2.31024123202e-08



3b.

(i) The estimated MSE under 10-fold CV for various alpha :

Estimated MSE of 10-fold CV (alpha=10) =  -0.0200342497979
Estimated MSE of 10-fold CV (alpha=1) =  -0.0197950213482
Estimated MSE of 10-fold CV (alpha=0.1) =  -0.01991972494
Estimated MSE of 10-fold CV (alpha=0.01) =  -0.0200559556275
Estimated MSE of 10-fold CV (alpha=0.001) =  -0.020089527318

(ii) The estimated MSE on the training data for various alpha :

Estimated MSE of training data (alpha=10) =  -0.000152996873221
Estimated MSE of training data (alpha=1) =  -8.73728120217e-05
Estimated MSE of training data (alpha=0.1) =  -1.63387410814e-05
Estimated MSE of training data (alpha=0.01) =  -1.31270726501e-06
Estimated MSE of training data (alpha=0.001) =  -2.31024123202e-08

(iii) Considering the MSE under both the cases, alpha=0.001 seems to be the best and the MSE is the least in that case.

(iv) This shows that the amount of overfitting reduces the quality of the result, ie. the prediction of crime in this case.

3c. Using polynomical_features to do quadratic regression.

In [150]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree=2)
poly.fit_transform(Xvalues)

from sklearn.pipeline import Pipeline
model = Pipeline([('poly', PolynomialFeatures(degree=2)),('linReg', linear_model.LinearRegression(fit_intercept=False))])
model = model.fit(Xvalues, YregVal)


yPolyPred=model.predict(Xvalues)
msePoly=cross_val_score(model,Xvalues,YregVal,cv=10,scoring='neg_mean_squared_error')
print("Estimated MSE under 10-fold CV = ",np.mean(msePoly))

msePoly=cross_val_score(model,Xvalues,yPolyPred,cv=10,scoring='neg_mean_squared_error')
print("Estimated MSE under training data = ",np.mean(msePoly))


Estimated MSE under 10-fold CV =  -0.129996952112
Estimated MSE under training data =  -0.129996952112


3c.

(i) Estimated MSE under 10-fold CV =  -0.129996952112

(ii) Estimated MSE under training data =  -0.129996952112

(iii) This model hasn't reduced the MSE from the linear regression model. So, the quadratic model is not better than linear regression model.

4a. Using the full data set for Decision Tree

In [163]:
import csv
from sklearn.preprocessing import Imputer
reader=csv.reader(open('C:/CrimePredictionData/communities-crime-full.csv','r'))
writer=csv.writer(open('C:/CrimePredictionData/outputSampleFull.csv','w',newline=''))
i=0
for row in reader:
    for i in range(len(row)):
        if(str(row[i])=='?'):
            row[i]=0
    if(i==0):
        row.append("highCrime")
        i=1
    if(row[127]!='ViolentCrimesPerPop'):
        if(float(row[127])>0.1):
            row.append("true")
        else:
            row.append("false")
    writer.writerow(row)


data=pd.read_csv('C:/CrimePredictionData/outputSampleFull.csv',header=0)

print ("Dataset Length: ", len(data))
print ("Dataset Shape:: ", data.shape)

Xvalues=data.values[:,5:127]
Yvalues=data.values[:,127]

for i in range(len(Yvalues)):
    Yvalues[i]=str(Yvalues[i])

X_train, X_test, Y_train, Y_test = train_test_split(Xvalues,Yvalues,test_size = 0.3)




Y_training=list()
for val in Y_train:
    Y_training.append(val)

#X_train=np.array(X_train).reshape(-1,1)
#Y_training=Y_training.reshape(-1,1)
y_training=np.asarray(Y_training)
y_training=y_training.reshape(-1,1)
dTreeClass=dTreeClass.fit(X_train,y_training)

#X_test=np.array(X_test).reshape((-1,1))
y_pred=dTreeClass.predict(X_test)
print(y_pred)

print("Accuracy = ",accuracy_score(Y_test,y_pred)*100,"%")
print("Precision = ",precision_score(Y_test,y_pred,pos_label="True")*100,"%")
print("Recall = ",recall_score(Y_test,y_pred,pos_label="True")*100,"%")

Dataset Length:  1994
Dataset Shape::  (1994, 128)
['True' 'True' 'True' 'False' 'False' 'False' 'True' 'True' 'True' 'False'
 'True' 'True' 'True' 'False' 'False' 'False' 'True' 'False' 'True' 'True'
 'True' 'True' 'False' 'False' 'False' 'True' 'False' 'True' 'True' 'False'
 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'False' 'True'
 'True' 'True' 'True' 'False' 'False' 'True' 'True' 'True' 'False' 'False'
 'False' 'False' 'True' 'True' 'True' 'True' 'True' 'True' 'False' 'False'
 'True' 'True' 'False' 'False' 'True' 'True' 'True' 'False' 'True' 'True'
 'True' 'False' 'True' 'True' 'True' 'False' 'False' 'True' 'True' 'False'
 'False' 'False' 'True' 'True' 'True' 'True' 'False' 'True' 'True' 'True'
 'True' 'True' 'True' 'True' 'False' 'True' 'True' 'True' 'False' 'False'
 'False' 'True' 'False' 'False' 'True' 'True' 'False' 'True' 'False' 'True'
 'True' 'False' 'True' 'True' 'True' 'True' 'False' 'True' 'True' 'False'
 'True' 'False' 'True' 'False' 'True' 'True' 'False' '

4a.

The CV results are worse, since the values are missing. This shows that missing values make it complex for the model to learn and predict.